In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from evodiff.pretrained import OA_DM_38M, OA_DM_640M
from evodiff.generate import generate_oaardm
from evodiff.conditional_generation import inpaint_simple 

torch.hub.set_dir('/data/users/kgeorge/workspace/evodiff')

/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
from DomainPrediction import BaseProtein
from DomainPrediction.eval import metrics
from DomainPrediction.utils import helper

In [4]:
protein = BaseProtein(file='../../Data/GxpS_ATC_AF.pdb')
T = [i for i in range(538,608)] ## 539-608

In [5]:
protein.get_residues(T) ## T domain

'GEIEIALATIWRELLNVEQVGRHDSFFALGGHSLLAVRMIERLRRIGLGLSVQTLFQHPTLSVLAQSLVP'

In [6]:
# checkpoint = OA_DM_38M()
checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint

In [14]:
model = model.cuda()

In [17]:
sequence = protein.sequence
start_idx = 538
end_idx = 608

seq_dict = {}
for i in range(100):
    sample, entire_sequence, generated_idr = inpaint_simple(model, sequence, start_idx, end_idx, tokenizer=tokenizer, device='cuda')
    id = f'EvoDiff_GxpS_ATC_gen_{i}'
    seq_dict[id] = entire_sequence


100%|██████████| 70/70 [00:22<00:00,  3.16it/s]


In [18]:
fasta_file = '/nethome/kgeorge/workspace/DomainPrediction/Data/evodiff_experiments/evodiff_100.fasta'
helper.create_fasta(seq_dict, fasta_file)

In [19]:
records = helper.read_fasta(fasta_file)
seq_dict_ = {}
for rec in records:
    seq_dict_[rec.id] = ''.join([str(rec.seq)[i] for i in T])

file = '/nethome/kgeorge/workspace/DomainPrediction/Data/evodiff_experiments/evodiff_100.T.fasta'
helper.create_fasta(seq_dict_, file)